## **Launch Sites Locations Analysis with Folium**

In [1]:
import folium
import wget
import pandas as pd

In [2]:
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

In [49]:
spacex_df = pd.read_csv("C:\\Users\\SHAARIF ANAS\\Desktop\\github\\SpaceX launch prediction\\DataSets\\SpaceX_2.csv")
spacex_df = spacex_df[['LaunchSite', 'Latitude', 'Longitude', 'Class']]
launch_sites_df = spacex_df.groupby(['LaunchSite'], as_index=False).first()
launch_sites_df = launch_sites_df[['LaunchSite', 'Latitude', 'Longitude']]
launch_sites_df

,LaunchSite,Latitude,Longitude
0,CCSFS SLC 40,28.561857,-80.577366
1,KSC LC 39A,28.608058,-80.603956
2,VAFB SLC 4E,34.632093,-120.610829


In [19]:
spacex_df.shape

(168, 5)

We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.

In [50]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]

In [51]:
# Initiate the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for i in range(3):
    coordinate = [launch_sites_df.iloc[i,1],launch_sites_df.iloc[i,2] ]
    launch_site = launch_sites_df.iloc[i,0]
    
    circle = folium.Circle(coordinate, radius=100, color='#d35400', fill=True).add_child(folium.Popup(launch_site))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
    marker = folium.map.Marker(
        coordinate,
    # Create an icon as a text label
        icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % launch_site,
        )
    )
    site_map.add_child(circle)
    site_map.add_child(marker)
    
site_map

### Success/failed launches for each site on the map

Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.

In [52]:
marker_cluster = MarkerCluster()

def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['Class'].apply(assign_marker_color)
spacex_df.tail(10)

,LaunchSite,Latitude,Longitude,Class,marker_color
158,CCSFS SLC 40,28.561857,-80.577366,1,green
159,CCSFS SLC 40,28.561857,-80.577366,1,green
160,KSC LC 39A,28.608058,-80.603956,1,green
161,VAFB SLC 4E,34.632093,-120.610829,1,green
162,CCSFS SLC 40,28.561857,-80.577366,1,green
163,KSC LC 39A,28.608058,-80.603956,1,green
164,VAFB SLC 4E,34.632093,-120.610829,1,green
165,CCSFS SLC 40,28.561857,-80.577366,1,green
166,CCSFS SLC 40,28.561857,-80.577366,1,green
167,KSC LC 39A,28.608058,-80.603956,1,green


In [53]:
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker = folium.Marker(
                location=[record['Latitude'], record['Longitude']],
                icon=folium.Icon(color='white', icon_color=record['marker_color'])
    )
    marker_cluster.add_child(marker)

site_map

### Distances between a Launch site and its proximities

Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)

In [54]:
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

We can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:

In [55]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [56]:
coastline_lat = 28.61169 
coastline_lon = -80.59725
distance_coastline = calculate_distance(launch_sites_df.iloc[1,1], launch_sites_df.iloc[1,2], coastline_lat, coastline_lon)
distance_coastline

0.7693764821302385

In [57]:
#Add a distance marker
coastline_coordinate = [coastline_lat, coastline_lon]
distance_marker = folium.Marker(
    coastline_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
        )
    )
site_map.add_child(distance_marker)

In [58]:
#Add a Polyline
coordinates = [(coastline_lat, coastline_lon),(launch_sites_df.iloc[1,1], launch_sites_df.iloc[1,2])]
lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

#### Closest Airport to KSC LC 39A

In [59]:
airport_lat = 28.51447
airport_long = -80.79732

distance_airport = calculate_distance(launch_sites_df.iloc[1,1], launch_sites_df.iloc[1,2], airport_lat, airport_long)

airport_coordinate = [airport_lat, airport_long]
distance_marker = folium.Marker(
    airport_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_airport),
        )
    )
site_map.add_child(distance_marker)

coordinates = [(airport_lat, airport_long),(launch_sites_df.iloc[1,1], launch_sites_df.iloc[1,2])]
lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

#### Closest city to VAFB SLC 4E

In [60]:
city_lat = 34.9525
city_long = -120.43642

distance_city = calculate_distance(launch_sites_df.iloc[2,1], launch_sites_df.iloc[2,2], city_lat, city_long)

city_coordinate = [city_lat, city_long]
distance_marker = folium.Marker(
    city_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_city),
        )
    )
site_map.add_child(distance_marker)

coordinates = [(city_lat, city_long),(launch_sites_df.iloc[2,1], launch_sites_df.iloc[2,2])]
lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)